In [25]:
import pandas as pd
data = pd.read_csv('preprocessed_data.csv')
data.head()

,temperature_2m,relative_humidity_2m,surface_pressure,cloud_cover,wind_speed_10m,weather_group,temperature_2m_lag1,surface_pressure_lag1,humidity_change,hour_sin,hour_cos,dew_point_gap,pressure_trend_6h,humidity_roll_mean_3h,month_sin,month_cos,temp_humidity_interaction,pressure_humidity_interaction
0,21.6,92,1014.4,100,4.3,Drizzle,22.9,1013.9,5.0,1.000000,6.123234e-17,1.4,-0.6,89.666667,0.5,0.866025,1987.2,93324.8
1,22.5,90,1014.9,100,2.9,Drizzle,21.6,1014.4,-2.0,0.965926,-2.588190e-01,1.6,0.2,89.666667,0.5,0.866025,2025.0,91341.0
2,23.5,89,1016.3,98,5.6,Drizzle,22.5,1014.9,-1.0,0.866025,-5.000000e-01,2.0,2.3,90.333333,0.5,0.866025,2091.5,90450.7
3,23.9,87,1016.5,83,6.4,Drizzle,23.5,1016.3,-2.0,0.707107,-7.071068e-01,2.3,2.9,88.666667,0.5,0.866025,2079.3,88435.5
4,24.8,82,1016.1,93,9.2,Drizzle,23.9,1016.5,-5.0,0.500000,-8.660254e-01,3.2,2.7,86.000000,0.5,0.866025,2033.6,83320.2


In [26]:
X = data.drop(columns=['weather_group'])
y = data['weather_group'].astype('category').cat.codes # Chuyển nhãn chữ sang số

In [28]:
X = data.drop(columns=['weather_group'])
y = data['weather_group'].astype('category').cat.codes

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42, shuffle=False)


In [31]:
from sklearn.preprocessing import RobustScaler
numeric_features = ['temperature_2m', 'relative_humidity_2m',
       'surface_pressure', 'cloud_cover', 'wind_speed_10m',
       'humidity_change', 'temperature_2m_lag1',
       'surface_pressure_lag1',
       'dew_point_gap',
       'pressure_trend_6h', 'humidity_roll_mean_3h']

robust_scaler = RobustScaler()
X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()
X_test_scaled = X_test.copy()
X_train_scaled[numeric_features] = robust_scaler.fit_transform(X_train[numeric_features])
X_val_scaled[numeric_features] = robust_scaler.transform(X_val[numeric_features])
X_test_scaled[numeric_features] = robust_scaler.transform(X_test[numeric_features])


## 4. Softmax Regression

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, PredefinedSplit, TimeSeriesSplit
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time
import numpy as np

### 4.1 Tìm kiếm siêu tham số (Hyperparameter Tuning)

Để tìm ra bộ tham số tối ưu cho mô hình Softmax Regression, chúng tôi sử dụng kỹ thuật **Grid Search** kết hợp với **Time Series Cross Validation**.

**Phương pháp:**
- **Grid Search**: Thử nghiệm exhaustive trên một lưới các giá trị tham số.
- **TimeSeriesSplit**: Vì dữ liệu là chuỗi thời gian, việc chia tập validation cần tôn trọng thứ tự thời gian. `TimeSeriesSplit` chia dữ liệu thành *k* fold liên tiếp, trong đó fold thứ *i* dùng làm train cho fold thứ *i+1* (validation), đảm bảo không dùng tương lai để dự đoán quá khứ.

**Các siêu tham số được tinh chỉnh:**
1.  **C** (Regularization Strength): [0.001, 0.01, 1, 10, 100]
2.  **solver**: ['lbfgs', 'newton-cg']
3.  **max_iter**: 1000


In [33]:
# Chuẩn bị dữ liệu: Gộp Train và Val để GridSearch với TimeSeriesSplit
X_train_val = pd.concat([X_train_scaled, X_val_scaled])
y_train_val = pd.concat([y_train, y_val])

# Tạo các biến tương tác quan trọng
X_train_val['temp_humidity_interaction'] = X_train_val['temperature_2m'] * X_train_val['relative_humidity_2m']
X_train_val['pressure_humidity_interaction'] = X_train_val['surface_pressure'] * X_train_val['relative_humidity_2m']

# Áp dụng tương tự cho X_test_scaled
X_test_scaled['temp_humidity_interaction'] = X_test_scaled['temperature_2m'] * X_test_scaled['relative_humidity_2m']
X_test_scaled['pressure_humidity_interaction'] = X_test_scaled['surface_pressure'] * X_test_scaled['relative_humidity_2m']

In [34]:
# Time Series Cross Validation
tscv = TimeSeriesSplit(n_splits=5)

# Thử nghiệm các tổ hợp trọng số
custom_weights = [{0: 1, 1: 1.2, 2: 1.5}, {0: 1, 1: 1.5, 2: 2.0}]

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'class_weight': [None, 'balanced'] + custom_weights,
    'solver': ['lbfgs', 'newton-cg']
}

softmax_reg = LogisticRegression(
    solver='lbfgs', 
    max_iter=1000
)

grid_search = GridSearchCV(
    estimator=softmax_reg,
    param_grid=param_grid,
    cv=tscv,
    scoring='f1_macro', # Dùng f1_macro để cân bằng
    n_jobs=-1,
    verbose=1
)

# Thực hiện tìm kiếm
start_search = time.time()
grid_search.fit(X_train_val, y_train_val)
end_search = time.time()

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best validation score: {grid_search.best_score_:.4f}")
print(f"Search time: {end_search - start_search:.2f}s")

best_model_softmax = grid_search.best_estimator_

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best parameters: {'C': 10, 'class_weight': {0: 1, 1: 1.5, 2: 2.0}, 'solver': 'lbfgs'}
Best validation score: 0.5091
Search time: 24.39s


In [ ]:
from sklearn.metrics import f1_score, make_scorer

# 1. Sửa danh sách trọng số tùy chỉnh để bao gồm cả trường hợp không dùng trọng số
custom_weights = [{0: 1, 1: 1.2, 2: 1.5}, {0: 1, 1: 1.5, 2: 2.5}]

# 2. Định nghĩa lưới tham số tập trung vào giảm Overfitting
param_grid = {
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10], # Thêm các giá trị nhỏ để giảm Overfitting
    'class_weight': [None, 'balanced'] + custom_weights, 
    'solver': ['lbfgs', 'newton-cg'],
    'max_iter': [2000] # Tăng số vòng lặp để đảm bảo hội tụ với C nhỏ
}

softmax_reg = LogisticRegression(solver='lbfgs', max_iter=2000)

# 4. Thiết lập GridSearchCV với f1_macro
# Lưu ý: Nên đổi scoring sang 'f1_macro' để so sánh công bằng với AdaBoost/RF
grid_search = GridSearchCV(
    estimator=softmax_reg,
    param_grid=param_grid,
    cv=tscv, 
    scoring='f1_macro', 
    n_jobs=-1,
    verbose=1
)

# 5. Thực hiện tìm kiếm
start_search = time.time()
grid_search.fit(X_train_val, y_train_val)
end_search = time.time()

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best validation score (F1-Macro): {grid_search.best_score_:.4f}")

best_model_softmax = grid_search.best_estimator_

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters: {'C': 1, 'class_weight': {0: 1, 1: 1.5, 2: 2.5}, 'max_iter': 2000, 'solver': 'lbfgs'}
Best validation score (F1-Macro): 0.5105


In [35]:
# Xem tầm quan trọng của các biến trong Softmax
for i, class_label in enumerate(best_model_softmax.classes_):
    print(f"\nTop features cho lớp {class_label}:")
    top_indices = np.argsort(np.abs(best_model_softmax.coef_[i]))[-5:]
    for idx in top_indices[::-1]:
        print(f"{X.columns[idx]}: {best_model_softmax.coef_[i][idx]:.4f}")


Top features cho lớp 0:
temperature_2m: 4.9585
temperature_2m_lag1: -4.5741
surface_pressure_lag1: -2.7352
dew_point_gap: 2.5463
surface_pressure: 2.0262

Top features cho lớp 1:
temperature_2m: 1.0797
relative_humidity_2m: -0.9855
surface_pressure_lag1: 0.8725
humidity_roll_mean_3h: 0.7191
temperature_2m_lag1: -0.6641

Top features cho lớp 2:
temperature_2m: -6.0382
temperature_2m_lag1: 5.2383
dew_point_gap: -2.8175
surface_pressure: -2.1218
surface_pressure_lag1: 1.8627


### 4.2 Kiểm thử mô hình (Model Evaluation)

Sau khi chọn được mô hình tốt nhất, chúng tôi tiến hành đánh giá trên tập kiểm thử (Test Set) hoàn toàn độc lập.

**Các chỉ số đánh giá (Metrics):**
1.  **Accuracy (Độ chính xác)**: Tỷ lệ số mẫu dự đoán đúng trên tổng số mẫu. Đo lường hiệu suất tổng quan.
2.  **Training Time & Inference Time**: 
    - *Training Time*: Thời gian huấn luyện lại mô hình tốt nhất.
    - *Inference Time*: Thời gian mô hình dự đoán trên tập test.
3.  **Precision (Độ chính xác của từng lớp)**: Trong số các mẫu được dự đoán là lớp A, bao nhiêu % thực sự là lớp A. Quan trọng khi muốn giảm thiểu báo động giả (False Positives).
4.  **Recall (Độ phủ)**: Trong số các mẫu thực sự là lớp A, bao nhiêu % được mô hình phát hiện đúng. Quan trọng khi muốn tránh bỏ sót (False Negatives).
5.  **F1-Score**: Trung bình điều hòa của Precision và Recall. Cân bằng giữa độ chính xác và độ phủ, hữu ích khi dữ liệu mất cân bằng.
6.  **Confusion Matrix**: Bảng ma trận hiển thị chi tiết sự nhầm lẫn giữa các lớp.

In [36]:
# Huấn luyện lại mô hình tốt nhất (hoặc sử dụng kết quả từ GridSearch đã fit trên Train+Val)
# Ở đây GridSearch refit=True mặc định sẽ fit lại trên toàn bộ dữ liệu đưa vào (Train + Val)
# Đo thời gian Inference
start_inf = time.time()
y_pred = best_model_softmax.predict(X_test_scaled)
end_inf = time.time()
inference_time = end_inf - start_inf

# Đánh giá trên Train và Test
train_acc = best_model_softmax.score(X_train_val, y_train_val)
test_acc = accuracy_score(y_test, y_pred)

print(f"Training Accuracy (Result of GridSearch refit): {train_acc:.4f}")
print(f"Testing Accuracy: {test_acc:.4f}")
print(f"Inference Time (on test set): {inference_time:.4f}s")

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

Training Accuracy (Result of GridSearch refit): 0.7793
Testing Accuracy: 0.6530
Inference Time (on test set): 0.0093s

Classification Report:

              precision    recall  f1-score   support

           0       0.77      0.65      0.70      1675
           1       0.57      0.68      0.62      1366
           2       0.61      0.60      0.60       472

    accuracy                           0.65      3513
   macro avg       0.65      0.64      0.64      3513
weighted avg       0.67      0.65      0.66      3513


Confusion Matrix:

[[1086  555   34]
 [ 292  925  149]
 [  35  154  283]]
